In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report
from minisom import MiniSom  # Thư viện cho Self-Organizing Maps (SOM)

# Bước 1: Load dữ liệu
df = pd.read_csv('/kaggle/input/model-data/Model_Data.csv')
X = df.drop(columns='label')
y = df['label']

# Bước 2: Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Bước 3: Chuẩn hóa dữ liệu
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

### Mô hình ANN + ANN
# Bước 4a: Xây dựng ANN đầu tiên để giảm dữ liệu
ann1 = MLPClassifier(hidden_layer_sizes=(100,), max_iter=300, random_state=42)
ann1.fit(X_train_scaled, y_train)
X_train_reduced = ann1.predict_proba(X_train_scaled)  # Dữ liệu đã giảm

# Bước 5a: Xây dựng ANN thứ hai cho dự đoán dựa trên dữ liệu đã giảm
ann2 = MLPClassifier(hidden_layer_sizes=(100,), max_iter=300, random_state=42)
ann2.fit(X_train_reduced, y_train)

# Bước 6a: Đánh giá mô hình ANN + ANN
X_test_reduced = ann1.predict_proba(X_test_scaled)
y_pred_ann = ann2.predict(X_test_reduced)
print("ANN + ANN Accuracy:", accuracy_score(y_test, y_pred_ann))
print("Classification Report for ANN + ANN:\n", classification_report(y_test, y_pred_ann))

### Mô hình SOM + ANN
# Bước 4b: Xây dựng SOM để giảm dữ liệu
som = MiniSom(10, 10, X_train_scaled.shape[1], sigma=1.0, learning_rate=0.5)
som.train_random(X_train_scaled, 100)

# Biểu diễn dữ liệu thành dữ liệu đại diện
X_train_reduced_som = [som.winner(x) for x in X_train_scaled]
X_test_reduced_som = [som.winner(x) for x in X_test_scaled]

# Chuyển đổi dữ liệu thành định dạng dễ hiểu cho ANN thứ hai
X_train_reduced_som = pd.get_dummies(X_train_reduced_som)
X_test_reduced_som = pd.get_dummies(X_test_reduced_som)
X_train_reduced_som, X_test_reduced_som = X_train_reduced_som.align(X_test_reduced_som, fill_value=0, axis=1)

# Bước 5b: Xây dựng ANN thứ hai cho dự đoán dựa trên dữ liệu SOM đã giảm
ann_som = MLPClassifier(hidden_layer_sizes=(100,), max_iter=300, random_state=42)
ann_som.fit(X_train_reduced_som, y_train)

# Bước 6b: Đánh giá mô hình SOM + ANN
y_pred_som = ann_som.predict(X_test_reduced_som)
print("SOM + ANN Accuracy:", accuracy_score(y_test, y_pred_som))
print("Classification Report for SOM + ANN:\n", classification_report(y_test, y_pred_som))
